In [1]:
# 1. Import necessary libraries
import pandas as pd
import geopandas as gpd
from shapely import wkb
from pathlib import Path

# 2. Read the CSV file containing trajectory data
csv_path = Path(r"\\tsclient\D\Siyu Zhao\data\Auckland region park\waitakere_trajectories.csv")


df = pd.read_csv(
    csv_path,
    sep=",",            
    header=0,
    dtype={
        "hashed_id": "string",
        "lat": "float64",          
        "lon": "float64",          
        "time": "int64",           
        "polygon_name": "category",
        "geom": "string"
    }
)

# 3. change unix_timestamp to datetime
df["datetime"] = pd.to_datetime(df["time"], unit="s", utc=True) 

# 4. Convert the WKB geometry column to a GeoDataFrame
df["geometry"] = df["geom"].apply(lambda x: wkb.loads(bytes.fromhex(x))) # Convert WKB hex string to Shapely geometry
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # Set the coordinate reference system to WGS 84

# 5. print
print(df.head()) # Display the first few rows of the DataFrame
print(df.columns.tolist()) # Display the list of columns in the DataFrame

                                  hashed_id        lat         lon  \
0  bb2389611301252b5a2cd60950a1cbd450dc0f64 -36.880467  174.549970   
1  6a564bcffb1e5fb79859602a62ff2d7800310894 -36.878074  174.549415   
2  267f9fef013128ad1dbbd27ddb75079653dc951d -36.880485  174.549969   
3  c2d4c716575356edd331b741ba572e63b6c2f6d5 -36.880668  174.549804   
4  267f9fef013128ad1dbbd27ddb75079653dc951d -36.880467  174.549947   

         time polygon_name                                               geom  \
0  1588691716            1  0101000020E6100000FF78AF5A99D16540871A8524B370...   
1  1605257908            1  0101000020E6100000D21DC4CE94D16540E4BB94BA6470...   
2  1600461195            1  0101000020E61000000B9A965899D165402DCF83BBB370...   
3  1605923541            1  0101000020E6100000B8E68EFE97D165401E51A1BAB970...   
4  1600940978            1  0101000020E61000001171732A99D16540871A8524B370...   

                   datetime                       geometry  
0 2020-05-05 15:15:16+00:00   P

In [3]:
from pyrosm import OSM
import numpy as np
np.float = np.float64 

# 6. Get the bounding box of the trajectory data
south, north = df["lat"].min(), df["lat"].max()  # latitude
west,  east  = df["lon"].min(), df["lon"].max()  # longitude
bbox = [west, south, east, north]  # bounding box               
print("bbox:", bbox)
osm = OSM(r"\\tsclient\D\Siyu Zhao\data\New Zealand OSM\new-zealand-latest.osm.pbf", bounding_box=bbox)



bbox: [174.444446, -37.050305, 174.635066, -36.868314]


In [4]:
# 7. Extract POIs related to travel and transportation


tourism_tags = {
    "tourism": [
        "attraction", "viewpoint",  "museum", "gallery",
         "sightseeing", "zoo", "theme_park", "aquarium", 
    ],
    "leisure": [
        "park", "nature_reserve", "garden", "playground", "picnic_site", "dog_park",
    ],
    "natural": [
        "waterfall", "peak", "beach", "spring", "volcano", 
        "cave_entrance" , "wetland", "bay" , "valley" ,
        "lake", "river", "stream",
    ],
    "sport" : [
        "surfing","scuba_diving","fishing","kayaking","rowing","rafting",
         "climbing","running","trail_running","mountain_biking","orienteering",
        "horse_riding","equestrian","paragliding","beachvolleyball","disc_golf"
    ],
    "amenity": [
        "restaurant", "cafe", "bar", "pub", "fast_food", "ice_cream", "toilets",
        "food_court","bbq","drinking_water","shower", "bench",
        "parking", "bicycle_parking", "ferry_terminal","fuel","bicycle_rental", "car_rental","car_sharing","taxi"
    ],
    "shop": [
        "convenience", "supermarket", "bakery", "mall",
        "outdoor","sports","surf","diving","bicycle","camping",
        "souvenir","gift","craft",
    ],
    "historic": [ "monument", "memorial","milestone", "heritage"],
                        
}

transport_tags = {
    "highway": [ "bus_stop","bus_station"],
    "public_transport": ["bus_stop", "ferry", "train_station","boarding_area"],
    "railway": ["subway","station"],
    "aeroway": ["aerodrome", "helipad"],
    "waterway": ["boat_ramp","dock"]
}

custom_filter = {**tourism_tags, **transport_tags} # Merge tourism and transport tags





In [ ]:
from shapely.geometry.base import BaseGeometry
from shapely.errors import WKTReadingError


# 8. Extract POIs from OSM data using the custom filter
raw_pois = osm.get_pois(custom_filter=custom_filter)







C:\Users\Siyu\AppData\Local\Temp\2\ipykernel_10300\1759372121.py:2: DeprecationWarning: WKTReadingError is deprecated and will be removed in a future version. Use ShapelyError instead (functions previously raising {name} will now raise a ShapelyError instead).
  from shapely.errors import WKTReadingError


815 POIs
          id   timestamp visible  version  \
0  267846635  1212322163   False        1   
1  281899623  1726726018   False        5   
2  343304706  1234068629   False        2   
3  364614727  1609091094   False        4   
4  364875376  1237783713   False        2   

                                                tags         lon  changeset  \
0                     {"created_by":"Potlatch 0.9c"}  174.541046        0.0   
1  {"brand":"Night 'n Day","brand:wikidata":"Q111...  174.619446        0.0   
2                    {"created_by":"Potlatch 0.10f"}  174.545837        0.0   
3  {"description":"Waterfall viewpoint","directio...  174.545029        0.0   
4                    {"created_by":"Potlatch 0.10f"}  174.458328        0.0   

         lat addr:city addr:housenumber  ... railway  \
0 -36.906548      None             None  ...    None   
1 -36.894032      None             None  ...    None   
2 -36.890720      None             None  ...    None   
3 -36.999802      Non

In [17]:
# Filter out invalid geometries
def is_valid_geom(g):
    return isinstance(g, BaseGeometry) and g.is_valid and not g.is_empty

pois = raw_pois[raw_pois["geometry"].apply(is_valid_geom)].copy()


# point_pois = pois[pois.geometry.type == "Point"]
# line_pois = pois[pois.geometry.type == "LineString"]
# polygon_pois = pois[pois.geometry.type == "Polygon"]


# print(len(point_pois), "Point POIs")
# print(len(line_pois), "Line POIs")
# print(len(polygon_pois), "Polygon POIs")
print(len(pois), "POIs")
print(pois.geometry.type.value_counts())

1711 POIs
Polygon            887
Point              815
MultiPolygon         5
MultiLineString      3
LineString           1
Name: count, dtype: int64


In [14]:
# 9. Count the number of POIs for each category


for k in ["amenity","tourism","leisure","natural","historic",
          "shop","highway","public_transport","railway","aeroway"]:
    if k in pois.columns:
        print(f"{k:<16}", pois[k].notna().sum())


amenity          801
tourism          68
leisure          360
natural          203
historic         13
shop             36
highway          226
public_transport 216
railway          3
aeroway          2


In [18]:
output_path = r"\\tsclient\D\Siyu Zhao\data\Auckland region park\Auckland-parks-poi.gpkg"
pois.to_file(output_path, driver="GPKG")

In [20]:
print(pois.head())

          id   timestamp visible  version  \
0  267846635  1212322163   False        1   
1  281899623  1726726018   False        5   
2  343304706  1234068629   False        2   
3  364614727  1609091094   False        4   
4  364875376  1237783713   False        2   

                                                tags         lon  changeset  \
0                     {"created_by":"Potlatch 0.9c"}  174.541046        0.0   
1  {"brand":"Night 'n Day","brand:wikidata":"Q111...  174.619446        0.0   
2                    {"created_by":"Potlatch 0.10f"}  174.545837        0.0   
3  {"description":"Waterfall viewpoint","directio...  174.545029        0.0   
4                    {"created_by":"Potlatch 0.10f"}  174.458328        0.0   

         lat addr:city addr:housenumber  ... railway  \
0 -36.906548      None             None  ...    None   
1 -36.894032      None             None  ...    None   
2 -36.890720      None             None  ...    None   
3 -36.999802      None        

In [21]:
print(print(pois.columns.tolist()))

['id', 'timestamp', 'visible', 'version', 'tags', 'lon', 'changeset', 'lat', 'addr:city', 'addr:housenumber', 'addr:housename', 'addr:postcode', 'addr:street', 'email', 'name', 'opening_hours', 'operator', 'phone', 'ref', 'website', 'tourism', 'zoo', 'leisure', 'outdoor_seating', 'natural', 'amenity', 'bicycle_parking', 'fountain', 'internet_access', 'parking', 'source', 'start_date', 'wikipedia', 'shop', 'historic', 'memorial', 'access', 'foot', 'highway', 'lit', 'public_transport', 'railway', 'geometry', 'osm_type', 'wetland', 'building', 'building:levels', 'drinking_water', 'area', 'surface', 'aeroway']
None
